In [162]:
import numpy as np
import pandas as pd
from scipy.stats import gamma
DATE = '2020_04_08'

In [163]:
data = pd.read_csv(f'countermeasures_features_{DATE}.csv').drop(columns='Unnamed: 0').fillna(0)


In [164]:
def gatherings(n):
    if n == np.nan:
        return 0
    elif n == 0:
        return 0
    return 1/(1+np.exp(-4+np.log(n)))


def curfew(n):
    if n < 1:
        return 0
    elif n<2:
        return 0.5
    return 1

def norm(maximum):
    def f(n):
        return n/maximum
    return f

def masks(n):
    if n==0:
        return np.nan
    return n/100
# def nj2us(s):
#     if s=='US:New Jersey':
#         return 'United States'
#     return s

REPORTCOLS1 = {'Gatherings limited to':('Gatherings banned',gatherings),
              'Business suspended':('Nonessential business suspension',norm(1)),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'General curfew':('Asymptomatic isolation - blanket',curfew),
              'Healthcare specialisation':('Healthcare specialisation',norm(7)),
              'Minor distancing and hygiene measures':('Social distancing and hygiene advice',norm(12)),
              'Phone line':('Phone hotline',norm(1)),
              'Mask wearing':('Mask wearing',masks),
              'Asymptomatic contact isolation':('Asymptomatic isolation - targeted',lambda x: x>=1)}

data0 = data.copy()

# US_STATES = data0['Country'][data0['Country'].str.startswith('US:')].unique()

# data0['Country'] = data0['Country'].apply(nj2us)
# data0.loc[data['Country']=='United States',
#                   data0.columns[:-1]] = data0.loc[data['Country'].isin(US_STATES),:].groupby('Date').mean()

# data0 = data0[~data0['Country'].str.startswith('US:')]

for rc, (vc, f) in REPORTCOLS1.items():
    data0[rc] = data0[vc].apply(f).astype(float)
    
outdata = data0[list(REPORTCOLS1.keys())+['Country','Date']]
outdata.to_csv(f'interventions_model_zeroto1_{DATE}.csv')

In [43]:
data0['Country'].unique()

array(['Austria', 'Germany', 'United Kingdom', 'South Korea', 'Singapore',
       'Israel', 'Japan', 'Sweden', 'San Marino', 'Slovenia', 'Canada',
       'Hong Kong', 'Taiwan', 'China', 'Italy', 'Czechia', 'Australia',
       'Romania', 'France', 'Portugal', 'Spain', 'Belgium', 'Luxembourg',
       'Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina',
       'Bulgaria', 'Denmark', 'Estonia', 'Cyprus', 'Croatia', 'Finland',
       'Georgia', 'Hungary', 'Latvia', 'Lithuania', 'Greece', 'Moldova',
       'Malta', 'Monaco', 'Netherlands', 'Iceland', 'Macedonia',
       'Ireland', 'Vatican City', 'Kazakhstan', 'Poland', 'Turkey',
       'Ukraine', 'Slovakia', 'Serbia', 'Switzerland', 'Norway',
       'Montenegro', 'Iran', 'Liechtenstein', 'Russia', 'Mexico', 'Egypt',
       'Palestine', 'Nepal', 'Iraq', 'South Africa', 'Namibia', 'Morocco',
       'Uganda'], dtype=object)

In [44]:
np.sqrt(2.5)

1.5811388300841898

In [45]:
np.sqrt(2)

1.4142135623730951

In [46]:
1.4/1.6

0.8749999999999999

In [161]:
gatherings(2),gatherings(100),gatherings(1000)

(0.9646631559719038, 0.35316172943491847, 0.05177152077445643)

In [166]:
q = outdata[outdata.columns[:-2]]

In [167]:
v = np.array([gamma.pdf(i,3,0,3) for i in range(1,16)])

In [168]:
v = v/v.sum()

In [169]:
np.array([i*p for i,p in enumerate(v)]).sum()

6.673588535539523

In [170]:
v

array([0.01492811, 0.04278584, 0.06897913, 0.08786792, 0.09837518,
       0.10150402, 0.09899471, 0.09264693, 0.08401779, 0.07432269,
       0.06443799, 0.0549483 , 0.04620762, 0.03839885, 0.03158492])

In [171]:
a = np.apply_along_axis(lambda m: np.convolve(m,v),arr=q,axis=0)

In [172]:
pd.DataFrame(a,columns=q.columns).corr()

,Gatherings limited to,Business suspended,Schools and universities closed,General curfew,Healthcare specialisation,Minor distancing and hygiene measures,Phone line,Mask wearing,Asymptomatic contact isolation
Gatherings limited to,1.000000,0.970167,0.935434,0.872089,0.651876,0.401228,0.293766,NaN,0.462904
Business suspended,0.970167,1.000000,0.896054,0.858473,0.684357,0.408356,0.250482,NaN,0.477891
Schools and universities closed,0.935434,0.896054,1.000000,0.862529,0.649073,0.443999,0.409257,NaN,0.534969
General curfew,0.872089,0.858473,0.862529,1.000000,0.773022,0.650881,0.560868,NaN,0.695792
Healthcare specialisation,0.651876,0.684357,0.649073,0.773022,1.000000,0.854936,0.654429,NaN,0.876664
Minor distancing and hygiene measures,0.401228,0.408356,0.443999,0.650881,0.854936,1.000000,0.755849,NaN,0.882521
Phone line,0.293766,0.250482,0.409257,0.560868,0.654429,0.755849,1.000000,NaN,0.706327
Mask wearing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Asymptomatic contact isolation,0.462904,0.477891,0.534969,0.695792,0.876664,0.882521,0.706327,NaN,1.000000


In [165]:
outdata['Business suspended'].unique()

array([0. , 0.5, 1. ])

In [174]:
outdata['General curfew'].unique()

array([0. , 0.5, 1. ])

In [175]:
outdata['Asymptomatic contact isolation'].unique()

array([0., 1.])